# Checking dbt Tables/Data

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

True

## Creating connection

In [2]:
db_url = os.getenv('DB_URL')
engine = create_engine(db_url)

## Bronze Layer

The idea here is to check some general aspects of the tables, to check which treatments need to be done and which filters I'm going to apply to the data to create the tables in the Silver layer. 

### Customers

In [29]:
df_customers = pd.read_sql('SELECT * FROM bronze_customers', con=engine)
display(df_customers.sample(10))

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
26936,c690c7dda7366ab3855de3bd40381f19,6645724f3e3ee27f91f12f122f879331,3572,sao paulo,SP
89594,e171def7e1de79d5479970d8629ad8c8,499577d71608181c4ac94226b315015e,60351,fortaleza,CE
66213,9eb585e566edc8b94a17ac5d6076d136,0e850ed5adc4abe0346435eda5b1a626,24220,niteroi,RJ
88834,045eecf8b7d258d4ac84816515328236,17cd440592c98a8dbbf7a99963274ffe,66023,belem,PA
45943,04ac504901720fa9836cbc6df1350e4e,a1da6c965fe2a3e566f6da53c09d0afd,80240,curitiba,PR
24208,aa86e106ec68532c2b965a16693440eb,92728d7e8084a741f4fcb49708a0b78d,28200,sao joao da barra,RJ
54301,d50dcc264b92c899c1a4f0f1c99cb67c,87bbba8720d775549bec72aaec36ff19,58074,joao pessoa,PB
90431,49c5edaa0c1b454a55e3803945367f3f,9ab1fa98e23aa9eaa9b285974d105e96,8685,suzano,SP
70586,50d6bcc25f2532c7a92f144a21187705,e4267ee795e0af3be8273761caef9bae,61610,caucaia,CE
34674,6c8a4da7b11b28b528bbb1561cd232f7,6b62ffc01d145a07072c3a54da59c5c9,49048,aracaju,SE


In [7]:
df_customers.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [8]:
df_customers.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [14]:
df_customers_numbers = pd.read_sql("""
  SELECT COUNT(*) AS tt_rows, 
    COUNT(DISTINCT customer_id) AS tt_customer_id,
    COUNT(DISTINCT customer_unique_id) AS tt_customer_unique_id
  FROM bronze_customers"""
  , con=engine)
display(df_customers_numbers)

,tt_rows,tt_customer_id,tt_customer_unique_id
0,99441,99441,96096


Above we can see that relationship between _customer_id_ and rows is 1:1, but the relationship between _customer_unique_id_ and _customer_id_ is 1:many.

Looking at the description of these columns in the table, the _customer_unique_id_ column represent the unique identifier of a customer, while the _customer_id_ is related to the orders table, where each order has a unique _customer_id_.

In [13]:
df_customers_check = pd.read_sql("""
  SELECT DISTINCT customer_unique_id, 
    COUNT(DISTINCT customer_id) AS tt_customer_ids
  FROM bronze_customers
  GROUP BY 1
  HAVING COUNT(DISTINCT customer_id) > 1
  ORDER BY 2 DESC"""
  , con=engine)
display(df_customers_check)

,customer_unique_id,tt_customer_ids
0,8d50f5eadf50201ccdcedfb9e2ac8455,17
1,3e43e6105506432c953e165fb2acf44c,9
2,1b6c7548a2a1f9037c1fd3ddfed95f33,7
3,6469f99c1f9dfae7733b25662e7f1782,7
4,ca77025e7201e3b30c44b472ff346268,7
...,...,...
2992,ff03923ad1eb9e32304deb7f9b2a45c9,2
2993,ff22e30958c13ffe219db7d711e8f564,2
2994,ff44401d0d8f5b9c54a47374eb48c1b8,2
2995,ff8892f7c26aa0446da53d01b18df463,2


Above we can see that there are ~3k customers (_customer_unique_id_) with 2 or more _customer_ids_. So, according to the documentation, these customers placed 2 or more orders (represented by _order_id_ in the orders table). 

### Category Translation

In [30]:
df_ctg_transl = pd.read_sql('SELECT * FROM bronze_category_translation', con=engine)
display(df_ctg_transl.sample(10))

,product_category_name,product_category_name_english
42,moveis_sala,furniture_living_room
15,telefonia_fixa,fixed_telephony
21,fashion_calcados,fashion_shoes
67,artes_e_artesanato,arts_and_craftmanship
49,construcao_ferramentas_seguranca,construction_tools_safety
13,tablets_impressao_imagem,tablets_printing_image
55,fashion_roupa_feminina,fashio_female_clothing
27,construcao_ferramentas_jardim,costruction_tools_garden
25,construcao_ferramentas_construcao,construction_tools_construction
35,casa_conforto,home_confort


In [18]:
df_ctg_transl.dtypes

product_category_name            object
product_category_name_english    object
dtype: object

In [19]:
df_ctg_transl.isna().sum()

product_category_name            0
product_category_name_english    0
dtype: int64

In [20]:
df_ctg_transl_numbers = pd.read_sql("""
  SELECT COUNT(*) AS tt_rows, 
    COUNT(DISTINCT product_category_name) AS tt_product_category,
    COUNT(DISTINCT product_category_name_english) AS tt_product_category_english
  FROM bronze_category_translation"""
  , con=engine)
display(df_ctg_transl_numbers)

,tt_rows,tt_product_category,tt_product_category_english
0,71,71,71


### Geolocation

In [31]:
df_geo = pd.read_sql('SELECT * FROM bronze_geolocation', con=engine)
display(df_geo.sample(10))

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
293029,13170,-22.821672,-47.276063,sumaré,SP
407162,20211,-22.912133,-43.202026,rio de janeiro,RJ
422216,21070,-22.842027,-43.276356,rio de janeiro,RJ
362437,16300,-21.411935,-50.078503,penapolis,SP
836259,79116,-20.427476,-54.636386,campo grande,MS
416529,20756,-22.887415,-43.307088,rio de janeiro,RJ
755320,64860,-7.231471,-44.557944,urucui,PI
484592,25225,-22.697538,-43.262965,duque de caxias,RJ
920302,89070,-26.872585,-49.130171,blumenau,SC
403293,19970,-22.794515,-50.218443,palmital,SP


In [23]:
df_geo.dtypes

geolocation_zip_code_prefix      int64
geolocation_lat                float64
geolocation_lng                float64
geolocation_city                object
geolocation_state               object
dtype: object

In [24]:
df_geo.isna().sum()

geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64

In [25]:
df_geo_numbers = pd.read_sql("""
  SELECT COUNT(*) AS tt_rows, 
    COUNT(DISTINCT geolocation_zip_code_prefix) AS tt_zip_codes,
    COUNT(DISTINCT geolocation_city) AS tt_citys,
    COUNT(DISTINCT geolocation_state) AS tt_states   
  FROM bronze_geolocation"""
  , con=engine)
display(df_geo_numbers)

,tt_rows,tt_zip_codes,tt_citys,tt_states
0,1000163,19015,8011,27


### Order Items

In [32]:
df_oi = pd.read_sql('SELECT * FROM bronze_order_items', con=engine)
display(df_oi.sample(10))

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
27678,3ef28988ea616750c5ac6a0058814056,1,a2b35668d3b4c6f34b35c9f702ecc5b9,20d83f3ef0e6925fd74bfd59170babf7,2017-08-20 21:30:13,39.90,17.63
103103,ea25a7c224e8ab2c05c2425da85c033d,1,01cf7c4cffff8db0a1cbe612bd2d50a4,f7720c4fa8e3aba4546301ab80ea1f1b,2018-08-21 23:10:13,23.90,18.32
102093,e7b8437c3c5f2441634448e5a2a3d8fb,1,ee6c49af1a2de55aa0e076257ab4ab99,1c68394e931a64f90ea236c5ea590300,2018-02-19 17:15:32,144.41,17.45
87205,c5fd2c7e23ffd8d1ca4c839b7c5596a3,1,aca2eb7d00ea1a7b8ebd4e68314663af,955fee9216a65b617aa5c0531780ce60,2017-12-05 03:18:33,75.00,20.02
65469,95a46ee84acaaabcaa312cd198120bc3,1,84da49736447aaa6ca16652f7e5a77d8,440dd6ab244315c632130ecfb63827b1,2018-05-23 00:31:08,84.90,13.95
52500,777e12368d3bb5d1b085cdbc60c6f910,1,43a39180da2ed07ba59252d23cf029e7,e5c84227854980f1db19a58958c551d9,2018-08-21 12:10:24,18.00,15.23
40314,5bc9327cbcfa76ec35110bc7c73c9fe6,1,c6dd917a0be2a704582055949915ab32,7a67c85e85bb2ce8582c35f2203ad736,2017-04-05 14:35:19,99.99,15.46
51040,73ff74f8f201c61c0c625cfe7ec63cd2,1,b9d2933371226d828303d9235b89369c,f0b47fbbc6dee9aafe415a6e33051b3f,2018-05-14 07:36:31,24.90,18.23
23626,35dfa74b22307b2c8543606b037b1685,1,241e398aacc909372622952b2ec6f954,de23c3b98a88888289c6f5cc1209054a,2017-11-23 10:46:57,179.90,13.60
94849,d71ddeb3600f197986283a88b7d360ca,1,5a883fb296ab66ac6a1e68e6ba2175c3,3d871de0142ce09b7081e2b9d1733cb1,2018-05-22 21:31:56,29.90,12.79


In [33]:
df_oi.dtypes

order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

In [34]:
df_oi.isna().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [35]:
df_oi_numbers = pd.read_sql("""
  SELECT COUNT(*) AS tt_rows, 
    COUNT(DISTINCT order_id) AS tt_order_id,
    COUNT(DISTINCT product_id) AS tt_product_id,
    COUNT(DISTINCT seller_id) AS tt_seller_id   
  FROM bronze_order_items"""
  , con=engine)
display(df_oi_numbers)

,tt_rows,tt_order_id,tt_product_id,tt_seller_id
0,112650,98666,32951,3095


Above we can see that the relationship between _order_id_ and the number of rows is 1:many. 

Looking at the documentation of the table, this is possible because 1 _order_id_ can have multiple _product_id_, and the number of products per order is represented in the _order_item_id_ column.

In [40]:
df_oi_check = pd.read_sql("""
  SELECT DISTINCT order_item_id, 
    COUNT(DISTINCT order_id) as tt_orders
  FROM bronze_order_items
  GROUP BY 1
  ORDER BY 1"""
  , con=engine)
display(df_oi_check)

,order_item_id,tt_orders
0,1,98666
1,2,9803
2,3,2287
3,4,965
4,5,460
5,6,256
6,7,58
7,8,36
8,9,28
9,10,25


The sum of the column _tt_orders_ from the table above is 112.650, which corresponds to the number of rows in the order items table.

### Order Payments

In [42]:
df_op = pd.read_sql('SELECT * FROM bronze_order_payments', con=engine)
display(df_op.sample(10))

,order_id,payment_sequential,payment_type,payment_installments,payment_value
87326,fc297415452e5dc0d06afcd16e864ca8,3,voucher,1,20.00
33330,4d41f2b0d7100876dcef7b92b955fabc,1,credit_card,1,89.55
22060,214d5b272bf8f9b576c175c02ce98f08,1,credit_card,1,57.77
35138,40b844d9eccc85cdb1d2ccef1efbeb40,1,boleto,1,207.49
9461,a43da589ba6d3c9eb959a36699088162,1,credit_card,1,18.89
102174,4245e49d026ef1f440041837b6eac52a,1,credit_card,1,67.36
97499,06d6f2f9ce499ed49d414ff36f583abf,1,credit_card,8,317.48
6743,28dfb1ca8926269a4cf22394487f6a83,1,credit_card,1,22.98
12284,b2f6d9c004d4df667af498c97106b8f1,1,credit_card,1,178.89
2648,c294004ec80f803b7835e397c4857e0e,1,credit_card,10,166.29


In [43]:
df_op.dtypes

order_id                 object
payment_sequential        int64
payment_type             object
payment_installments      int64
payment_value           float64
dtype: object

In [44]:
df_op.isna().sum()

order_id                0
payment_sequential      0
payment_type            0
payment_installments    0
payment_value           0
dtype: int64

In [45]:
df_op_numbers = pd.read_sql("""
  SELECT COUNT(*) AS tt_rows, 
    COUNT(DISTINCT order_id) AS tt_order_id
  FROM bronze_order_payments"""
  , con=engine)
display(df_op_numbers)

,tt_rows,tt_order_id
0,103886,99440


Above we can see that the relationship between _order_id_ and the number of rows is 1:many. 

Looking at the documentation of the table, this is possible because 1 _order_id_ can have more than one _payment_type_. If an order has more than one payment method, this is tracked in the _payment_sequential_ column. Here is an example: 

In [87]:
df_op_example = pd.read_sql("""
  SELECT *
  FROM bronze_order_payments
  WHERE order_id = 'b80ba0f0c3d2a63feac0116f533cc6a9'
  ORDER BY 2 ASC"""
  , con=engine)
display(df_op_example)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b80ba0f0c3d2a63feac0116f533cc6a9,1,credit_card,1,11.10
1,b80ba0f0c3d2a63feac0116f533cc6a9,2,voucher,1,19.00
2,b80ba0f0c3d2a63feac0116f533cc6a9,3,voucher,1,2.99


### Order Reviews

In [46]:
df_or = pd.read_sql('SELECT * FROM bronze_order_reviews', con=engine)
display(df_or.sample(10))

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
37166,bc466002a6d8460ad53dc3265a5b9c25,45aaf4069d08b57b6a114f06191ce5fe,5,None,None,2018-05-26 00:00:00,2018-05-26 16:20:30
91925,4a98249a9372af92649ec163a526b227,e35ad21539b5000d5f10bff2b242a66f,4,None,None,2018-03-14 00:00:00,2018-03-17 13:28:17
44193,5593bc03e3f0c2d27afc424b991898b7,6dfdc1eefc7a125d1bb81feb04a6c2e6,4,None,None,2017-02-19 00:00:00,2017-02-19 19:17:51
20013,1eb7a0040532e7ec9971202a986b82c6,5278efafa60c7005d03202b122eaea55,1,None,Recebi o skate com duas avarias.\r\nO kit de s...,2017-08-24 00:00:00,2017-08-26 02:28:14
68644,129a548e881a28ae3707e19c3d21d3f1,b667484df2bb8bef1e61788f1c7556f5,5,None,Atendimento rápido sem burocracia fiz minha co...,2018-01-17 00:00:00,2018-01-18 23:09:47
68456,cf92878dc4153ee8933d281f22a5a4cb,afd3ddd9caf7a0969cc51740136d4a4f,3,None,None,2017-08-15 00:00:00,2017-08-16 15:49:34
83270,be9051cb1695f81c26a7ae23d87c328d,a9f75df1cef311cb1a83706ed6992b41,5,None,None,2018-01-27 00:00:00,2018-01-28 06:55:30
86290,b36cf1c8bede354ef191a052cad98581,ccd7d7de58c07e32a30717c6d7f5949d,5,None,None,2017-10-25 00:00:00,2017-10-25 18:18:34
59884,046045f9742036ad9b52c19b4e112697,4f9b6a9b3216925859301fc767b0fece,3,None,O produto é muito bom! As cores e a combinação...,2018-04-15 00:00:00,2018-04-23 13:13:40
50940,80ece1f2521d4d61c85bbfcef646e607,39c31edc53ca82584b62224bcbce3801,1,None,Não tem como avaliar positivamente se o produt...,2017-08-23 00:00:00,2017-08-26 18:02:04


In [47]:
df_or.dtypes

review_id                  object
order_id                   object
review_score                int64
review_comment_title       object
review_comment_message     object
review_creation_date       object
review_answer_timestamp    object
dtype: object

In [48]:
df_or.isna().sum()

review_id                      0
order_id                       0
review_score                   0
review_comment_title       87656
review_comment_message     58247
review_creation_date           0
review_answer_timestamp        0
dtype: int64

Above we can see that are a lot of reviews without a title or a comment. The table documentation doesn't mention it, but I'm going to assume that these are not mandatory when answering the satisfaction survey.

In [49]:
df_or_numbers = pd.read_sql("""
  SELECT COUNT(*) AS tt_rows, 
    COUNT(DISTINCT review_id) AS tt_review_id,
    COUNT(DISTINCT order_id) AS tt_order_id,
    MIN(review_creation_date) as min_review_creation_date, 
    MAX(review_creation_date) as max_review_creation_date, 
    MIN(review_answer_timestamp) as min_review_answer_date, 
    MAX(review_answer_timestamp) as max_review_answer_date
  FROM bronze_order_reviews"""
  , con=engine)
display(df_or_numbers)

,tt_rows,tt_review_id,tt_order_id,min_review_creation_date,max_review_creation_date,min_review_answer_date,max_review_answer_date
0,99224,98410,98673,2016-10-02 00:00:00,2018-08-31 00:00:00,2016-10-07 18:32:28,2018-10-29 12:27:35


Again we can see that the relationship between _review_id_ and the number of rows is 1:many. 

The table documentation doesn't mention if this is a expected behavior, so I'm going to investigate a litte bit more. 

In [51]:
df_or_check = pd.read_sql("""
  SELECT DISTINCT order_id, 
    COUNT(DISTINCT review_id) as tt_reviews
  FROM bronze_order_reviews
  GROUP BY 1 
  HAVING COUNT(DISTINCT review_id) > 1
  ORDER BY 2 DESC"""
  , con=engine)
display(df_or_check)

,order_id,tt_reviews
0,03c939fd7fd3b38f8485a0f95798f1f6,3
1,8e17072ec97ce29f0e1f111e598b0c85,3
2,c88b1d1b157a9999ce368f218a407141,3
3,df56136b8031ecd28e200bb18e6ddb2e,3
4,0035246a40f520710769010f752e7507,2
...,...,...
542,fd95ae805c63c534f1a64589e102225e,2
543,fe041ba1c9f54016432fa6ee91709dbc,2
544,ff763b73e473d03c321bcd5a053316e8,2
545,ff850ba359507b996e8b2fbb26df8d03,2


Above we can see that are orders with more than one review, so I'm going to assume that this happens when the order has more than one product, or products from different sellers. 

In [ ]:
df_or_check_2 = pd.read_sql("""
  SELECT DISTINCT review_id, 
    COUNT(*) as tt_rows
  FROM bronze_order_reviews
  GROUP BY 1 
  HAVING COUNT(*) > 1
  ORDER BY 2 DESC"""
  , con=engine)
display(df_or_check_2)

,review_id,tt_rows
0,08528f70f579f0c830189efc523d2182,3
1,0c76e7a547a531e7bf9f0b99cba071c1,3
2,1fb4ddc969e6bea80e38deec00393a6f,3
3,2172867fd5b1a55f98fe4608e1547b4b,3
4,2d6ac45f859465b5c185274a1c929637,3
...,...,...
784,fde2e6abaf5bb64f7407a44741c24dec,2
785,fde5986d35c89aa1b6ce4149de82a0d3,2
786,fe5c833752953fed3209646f1f63b53c,2
787,ff2fc9e68f8aabfbe18d710b83aabd30,2


But above we can see that are triplicated/duplicated _review_id_, which I think is strange. Let's check an example: 

In [53]:
df_or_example = pd.read_sql("""
  SELECT *
  FROM bronze_order_reviews
  WHERE review_id = '08528f70f579f0c830189efc523d2182'"""
  , con=engine)
display(df_or_example)

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,08528f70f579f0c830189efc523d2182,7813842ae95e8c497fc0233232ae815a,1,Produto errado,Entrega do produto diferente do solicitado\r\n...,2018-08-03 00:00:00,2018-08-06 00:09:52
1,08528f70f579f0c830189efc523d2182,03310aa823a66056268a3bab36e827fb,1,Produto errado,Entrega do produto diferente do solicitado\r\n...,2018-08-03 00:00:00,2018-08-06 00:09:52
2,08528f70f579f0c830189efc523d2182,53c71d3953507c6239ff73917ed358c9,1,Produto errado,Entrega do produto diferente do solicitado\r\n...,2018-08-03 00:00:00,2018-08-06 00:09:52


The same _review_id_ is related to different _order_id_. The score, the comment title, the comment itself and the dates are the same, only the _order_id_ is different. 

I'm going to assume this is an expected product/data behavior, but it's something to take note and document it later in the project.

### Orders

In [55]:
df_orders = pd.read_sql('SELECT * FROM bronze_orders', con=engine)
display(df_orders.sample(10))

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
52148,c79f6f90ae7d5b3b63033215b568c827,a2275d6d5cd707a2bdc00b70b8098109,delivered,2017-10-27 05:10:43,2017-10-27 05:25:59,2017-11-07 15:19:14,2017-11-13 18:48:42,2017-11-16 00:00:00
87166,4940ddf2b7091647a772152ed1de4a2b,c24638e1ff5efad8e21f689d7a1f0c6d,delivered,2018-08-11 18:52:49,2018-08-11 19:05:17,2018-08-13 09:03:00,2018-08-21 23:26:27,2018-08-31 00:00:00
66839,6004599e0ca050d5d2e6b6cc27616061,34c14391e34ba77edd322046b9a1eee5,delivered,2018-06-04 15:54:29,2018-06-06 03:11:18,2018-06-06 12:21:00,2018-06-21 17:54:26,2018-07-13 00:00:00
9654,4aaf9b9778e313ecfcfe11ba5673b136,a0af680b53dd9ddc1df27ee4a01fe25e,delivered,2018-03-24 15:11:38,2018-03-25 15:10:24,2018-03-26 19:31:01,2018-06-07 13:28:40,2018-04-27 00:00:00
96204,32a0d90ee681f59e353de7e3a1b46fb4,0548553d5a48db3e51de97645681dff4,delivered,2018-01-17 15:35:49,2018-01-17 15:49:00,2018-01-18 14:59:00,2018-01-23 21:04:48,2018-02-07 00:00:00
49181,15d8198b8e9c97e5e6a5007b0bb1550f,7d8c69b15f2b331d4aceffd9672c8029,delivered,2017-04-25 21:06:19,2017-04-25 21:15:12,2017-04-27 11:59:50,2017-05-08 17:16:52,2017-05-24 00:00:00
89713,787948cceabfd5a5dfde3e8d693e0303,c2b0ab58b7187c26170b62adcb523f87,delivered,2017-11-05 12:08:40,2017-11-05 12:26:22,2017-11-07 10:36:56,2017-12-04 23:48:40,2017-12-19 00:00:00
82673,a3e9e5209110a0dcfe5b20b01e6a69e7,cb6b298a2a15ea89e712248ae05dbab4,delivered,2018-07-14 10:21:43,2018-07-17 06:31:00,2018-07-17 11:58:00,2018-08-02 01:36:35,2018-08-07 00:00:00
28091,7ce9cf4a278e42edc813df45af48caf3,cecf2800e753cd1de4e002dd30874adb,delivered,2018-06-27 11:26:01,2018-06-27 11:35:17,2018-06-28 10:08:00,2018-07-03 22:03:18,2018-07-23 00:00:00
89646,0332002e367c17e6c4111ae2dd8a6aaf,6e790a587e88f8751cf58bd0f45d8e15,delivered,2018-03-08 20:06:30,2018-03-10 03:49:27,2018-03-19 22:17:38,2018-03-26 20:39:57,2018-04-11 00:00:00


In [56]:
df_orders.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [57]:
df_orders.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

Above we can see that orders with some _null_ values in the columns that are related to dates. This can be explained by orders placed but not approved/delivered yet, but I'm going to investigate it.

In [59]:
df_orders_numbers = pd.read_sql("""
  SELECT COUNT(*) AS tt_rows, 
    COUNT(DISTINCT order_id) AS tt_order_id,
    COUNT(DISTINCT customer_id) AS tt_customer_id,
    MIN(order_purchase_timestamp) AS min_purchase_ts, 
    MAX(order_purchase_timestamp) AS max_purchase_ts, 
    MIN(order_approved_at) AS min_approved_at, 
    MAX(order_approved_at) AS max_approved_at,
    MIN(order_delivered_carrier_date) AS min_delivered_carrier_dt, 
    MAX(order_delivered_carrier_date) AS max_delivered_carrier_dt,
    MIN(order_delivered_customer_date) AS min_delivered_dt, 
    MAX(order_delivered_customer_date) AS max_delivered_dt,
    MIN(order_estimated_delivery_date) AS min_estimated_delivered_dt, 
    MAX(order_estimated_delivery_date) AS max_estimated_delivered_dt                                
  FROM bronze_orders"""
  , con=engine)
display(df_orders_numbers)

,tt_rows,tt_order_id,tt_customer_id,min_purchase_ts,max_purchase_ts,min_approved_at,max_approved_at,min_delivered_carrier_dt,max_delivered_carrier_dt,min_delivered_dt,max_delivered_dt,min_estimated_delivered_dt,max_estimated_delivered_dt
0,99441,99441,99441,2016-09-04 21:15:19,2018-10-17 17:30:18,2016-09-15 12:16:38,2018-09-03 17:40:06,2016-10-08 10:34:01,2018-09-11 19:48:28,2016-10-11 13:46:32,2018-10-17 13:22:46,2016-09-30 00:00:00,2018-11-12 00:00:00


In [68]:
df_orders_example = pd.read_sql("""
  SELECT DISTINCT order_status, 
    COUNT(DISTINCT order_id) as tt_orders_id                              
  FROM bronze_orders
  WHERE order_approved_at IS NULL
  GROUP BY 1
  ORDER BY 2 DESC"""
  , con=engine)
display(df_orders_example)

,order_status,tt_orders_id
0,canceled,141
1,delivered,14
2,created,5


As expected, the majority of orders where the approval date is null have a 'canceled' or 'created' _order_status_, which makes sense. But there are a few orders with a 'delivered' _order_status_, which doesn't makes sense, so I'm going to remove these orders in the silver layer.

In [70]:
df_orders_example_2 = pd.read_sql("""
  SELECT DISTINCT order_status, 
    COUNT(DISTINCT order_id) as tt_orders_id                          
  FROM bronze_orders
  WHERE order_delivered_customer_date IS NULL
  GROUP BY 1
  ORDER BY 2 DESC"""
  , con=engine)
display(df_orders_example_2)

,order_status,tt_orders_id
0,shipped,1107
1,canceled,619
2,unavailable,609
3,invoiced,314
4,processing,301
5,delivered,8
6,created,5
7,approved,2


Same thing here, the majority of orders where the delivery date for the customer is null have an appropriate _order_status_. But again there few orders with a 'delivered' _order_status_, which doesn't makes sense, so I'm going to remove these orders in the silver layer. 

In [71]:
df_orders_example_3 = pd.read_sql("""
  SELECT DISTINCT order_status, 
    COUNT(DISTINCT order_id) as tt_orders_id                          
  FROM bronze_orders
  WHERE order_delivered_carrier_date IS NULL
  GROUP BY 1
  ORDER BY 2 DESC"""
  , con=engine)
display(df_orders_example_3)

,order_status,tt_orders_id
0,unavailable,609
1,canceled,550
2,invoiced,314
3,processing,301
4,created,5
5,approved,2
6,delivered,2


Same thing here, but looking at the date the product was delivered to the carrier (logistic partner responsible for the delivery to the customer).

### Products

In [61]:
df_products = pd.read_sql('SELECT * FROM bronze_products', con=engine)
display(df_products.sample(10))

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
12464,9f1330d9d3ea820ad8e4d0d58b7f08cd,eletroportateis,45.0,219.0,2.0,50.0,16.0,8.0,11.0
29370,2cb44978a2f8ca4e770be12e5491ac32,construcao_ferramentas_construcao,39.0,1087.0,1.0,950.0,22.0,22.0,16.0
13295,6156141755987eaf93bc9b959601db45,bebes,50.0,331.0,1.0,13800.0,65.0,35.0,55.0
12131,e80a0c4b0bb3f2f171f0fe8360513d90,fashion_roupa_masculina,34.0,185.0,1.0,225.0,19.0,27.0,27.0
25646,6a96ef77fb9fd6a8de17df8d3b5a3087,esporte_lazer,54.0,618.0,2.0,5700.0,82.0,16.0,26.0
12557,e9e8ec421793c31261cde20786e9ff16,malas_acessorios,27.0,512.0,6.0,5000.0,50.0,45.0,25.0
23587,aa53b7499856e6df13008f34100cc336,moveis_decoracao,53.0,629.0,1.0,1800.0,30.0,35.0,30.0
29419,9b609e296b89940552cdce39983b2f71,telefonia,46.0,879.0,1.0,100.0,18.0,6.0,11.0
30109,3c85232d9c97c6784684288a275f48e0,moveis_decoracao,55.0,563.0,1.0,300.0,35.0,10.0,25.0
29793,bc8b883240abcf055de69c37170844d7,fashion_bolsas_e_acessorios,48.0,1359.0,1.0,200.0,16.0,2.0,11.0


In [62]:
df_products.dtypes

product_id                     object
product_category_name          object
product_name_lenght           float64
product_description_lenght    float64
product_photos_qty            float64
product_weight_g              float64
product_length_cm             float64
product_height_cm             float64
product_width_cm              float64
dtype: object

In [63]:
df_products.isna().sum()

product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

In [73]:
df_prd_numbers = pd.read_sql("""
  SELECT COUNT(*) AS tt_rows, 
    COUNT(DISTINCT product_id) AS tt_product_id                         
  FROM bronze_products"""
  , con=engine)
display(df_prd_numbers)

,tt_rows,tt_product_id
0,32951,32951


In [75]:
df_prd_example = pd.read_sql("""
  SELECT *                        
  FROM bronze_products
  WHERE product_category_name IS NULL"""
  , con=engine)
display(df_prd_example.sample(10))

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
574,baded889837476e342ac1e93c87ce383,None,None,None,None,200.0,16.0,2.0,11.0
105,9df852a6c839bc3202227666d70c7cae,None,None,None,None,9150.0,49.0,32.0,35.0
307,69c2f05e34d543dc71cfe745bfac8499,None,None,None,None,650.0,30.0,3.0,30.0
8,7af3e2da474486a3519b0cba9dea8ad9,None,None,None,None,200.0,22.0,14.0,14.0
179,9246752373bd625f47da5e3d8b41b1dc,None,None,None,None,700.0,20.0,21.0,20.0
283,eaecac42c42a65691bc3662b123e884d,None,None,None,None,100.0,25.0,2.0,17.0
507,de6702c748c719cbd13bb814f006ba22,None,None,None,None,150.0,16.0,9.0,11.0
282,c638e232871cb1e760eaa7e724bc3eb2,None,None,None,None,300.0,20.0,17.0,13.0
537,dc8128d60372ee4869b22114e3bc4957,None,None,None,None,200.0,25.0,5.0,15.0
546,88c09dd121ebe0b119a759773d405cc4,None,None,None,None,300.0,22.0,9.0,16.0


Above we have some _product_id_ that don't have a _product_category_name_ value, but I think this won't be a problem later on. 

### Sellers

In [77]:
df_sell = pd.read_sql('SELECT * FROM bronze_sellers', con=engine)
display(df_sell.sample(10))

,seller_id,seller_zip_code_prefix,seller_city,seller_state
1929,1f867308a0ce13133a4e639ff4de0c02,82510,curitiba,PR
2225,fffd5413c0700ac820c7069d66d98c89,13908,amparo,SP
1524,70126eecc6aa1274392a1743866e9678,6449,barueri,SP
1829,18e694e0e48ed6f7aa3f24aade5fd697,13224,varzea paulista,SP
1218,25e6ffe976bd75618accfe16cefcbd0d,5503,sao paulo,SP
1470,e2a1ac9bf33e5549a2a4f834e70df2f8,1210,sao paulo,SP
2479,634964b17796e64304cadf1ad3050fb7,21840,rio de janeiro,RJ
1687,9485a23c0430ba1d58d359391606fce6,9015,santo andre,SP
939,2fb25ab44bdbeae6a4d816e0d0c9d500,27274,volta redonda,RJ
2020,695051e2981c8b19e77cafe450d5bdf3,6276,osasco,SP


In [78]:
df_sell.dtypes

seller_id                 object
seller_zip_code_prefix     int64
seller_city               object
seller_state              object
dtype: object

In [79]:
df_sell.isna().sum()

seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64

In [80]:
df_sell_numbers = pd.read_sql("""
  SELECT COUNT(*) AS tt_rows, 
    COUNT(DISTINCT seller_id) AS tt_seller_id                         
  FROM bronze_sellers"""
  , con=engine)
display(df_sell_numbers)

,tt_rows,tt_seller_id
0,3095,3095
